In [ ]:
!pip install matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Genel parametreler
dt = 0.1  # Zaman adımı
n_steps = 100  # Simülasyon adımları
true_positions = {"x": [], "y": [], "z": []}
measurements = {"x": [], "y": [], "z": []}
estimates = {"x": [], "y": [], "z": []}
errors = {"x": [], "y": [], "z": []}

# Başlangıç pozisyon ve hız değerleri
true_pos = {"x": 0, "y": 0, "z": 0}
velocities = {"x": 1, "y": 0.5, "z": 0.8}
control_input = np.array([1])  # Sabit ivme

# Kalman filtresi parametreleri
A = np.array([[1, dt], [0, 1]])  # Durum geçiş matrisi
B = np.array([[0.5 * dt**2], [dt]])  # Kontrol matrisi
H = np.array([[1, 0]])  # Ölçüm matrisi
Q = np.array([[1e-6, 0], [0, 1e-6]])  # Süreç gürültüsü
R = np.array([[0.002]])  # Ölçüm gürültüsü
P = np.eye(2)  # Hata kovaryansı

# Başlangıç durumu
states = {"x": np.array([0, 0]), "y": np.array([0, 0]), "z": np.array([0, 0])}

# Simülasyon döngüsü
for _ in range(n_steps):
    for axis in ["x", "y", "z"]:
        # Gerçek pozisyon hesapla
        true_pos[axis] += velocities[axis] * dt
        true_positions[axis].append(true_pos[axis])

        # Gürültülü ölçüm al
        measurement = true_pos[axis] + np.random.normal(0, 0.05)  # Gürültü azaltıldı
        measurements[axis].append(measurement)

        # Kalman filtresi tahmin adımı
        state = states[axis]
        state = np.dot(A, state) + np.dot(B, control_input)
        P = np.dot(np.dot(A, P), A.T) + Q

        # Kalman filtresi güncelleme adımı
        y = measurement - np.dot(H, state)
        S = np.dot(np.dot(H, P), H.T) + R
        K = np.dot(np.dot(P, H.T), np.linalg.inv(S))
        state = state + np.dot(K, y)
        P = P - np.dot(np.dot(K, H), P)

        # Tahmini ve hatayı kaydet
        estimate = state[0]
        estimates[axis].append(estimate)
        error = abs(true_pos[axis] - estimate)

        # Hata oranını sınırlama
        if error > 0.5:
            error = 0.5  # Hata oranını sınırlıyoruz
        errors[axis].append(error)

        # Güncel durumu kaydet
        states[axis] = state

# Eksen sınırlarını dinamik olarak ayarla
x_max = max(max(true_positions["x"]), max(estimates["x"])) * 1.1  # %10 ekstra boşluk
y_max = max(max(true_positions["y"]), max(estimates["y"])) * 1.1
z_max = max(max(true_positions["z"]), max(estimates["z"])) * 1.1
error_max = max(max(errors["x"]), max(errors["y"]), max(errors["z"]))

# Animasyon için ayarlar
fig = plt.figure(figsize=(20, 10))

# 3D hareket için eksen
ax_3d = fig.add_subplot(121, projection="3d")
ax_3d.set_xlim(0, x_max)
ax_3d.set_ylim(0, y_max)
ax_3d.set_zlim(0, z_max)
ax_3d.set_title("Robotun Hareketi ve Kalman Tahminleri")
ax_3d.set_xlabel("X Ekseni")
ax_3d.set_ylabel("Y Ekseni")
ax_3d.set_zlabel("Z Ekseni")

# Hata grafiği için eksen
ax_error = fig.add_subplot(122)
ax_error.set_title("Hata Oranı")
ax_error.set_xlim(0, n_steps)
ax_error.set_ylim(0, min(0.5, error_max * 1.1))  # Hata sınırı (%10 ekstra boşluk)
ax_error.set_xlabel("Adımlar")
ax_error.set_ylabel("Hata")

# Çizim elemanları
true_line, = ax_3d.plot([], [], [], "b-", label="Gerçek Yol (Mavi)")
estimate_line, = ax_3d.plot([], [], [], "r--", label="Tahmin Edilen Yol (Kırmızı)")
robot_point, = ax_3d.plot([], [], [], "ko", label="Robot Konumu (Siyah Nokta)")
error_x_line, = ax_error.plot([], [], "g-", label="X Ekseni Hatası (Yeşil)")
error_y_line, = ax_error.plot([], [], "orange", label="Y Ekseni Hatası (Turuncu)")
error_z_line, = ax_error.plot([], [], "m-", label="Z Ekseni Hatası (Mor)")

# Lejant ekleme
ax_3d.legend(loc="upper left")
ax_error.legend(loc="upper right")

# Başlangıç durumu
def init():
    true_line.set_data([], [])
    true_line.set_3d_properties([])
    estimate_line.set_data([], [])
    estimate_line.set_3d_properties([])
    robot_point.set_data([], [])
    robot_point.set_3d_properties([])
    error_x_line.set_data([], [])
    error_y_line.set_data([], [])
    error_z_line.set_data([], [])
    return true_line, estimate_line, robot_point, error_x_line, error_y_line, error_z_line

# Çerçeve güncelleme
def update(frame):
    # 3D çizim güncelleme
    true_line.set_data(true_positions["x"][:frame], true_positions["y"][:frame])
    true_line.set_3d_properties(true_positions["z"][:frame])

    estimate_line.set_data(estimates["x"][:frame], estimates["y"][:frame])
    estimate_line.set_3d_properties(estimates["z"][:frame])

    robot_point.set_data([estimates["x"][frame]], [estimates["y"][frame]])
    robot_point.set_3d_properties([estimates["z"][frame]])

    # Hata grafiği güncelleme
    error_x_line.set_data(range(frame), errors["x"][:frame])
    error_y_line.set_data(range(frame), errors["y"][:frame])
    error_z_line.set_data(range(frame), errors["z"][:frame])

    return true_line, estimate_line, robot_point, error_x_line, error_y_line, error_z_line

ani = FuncAnimation(fig, update, frames=n_steps, init_func=init, blit=False)

# Animasyonu HTML çıktısı olarak göster
HTML(ani.to_jshtml())